In [38]:
%load_ext autoreload
%autoreload 2

import numpy as np
import subprocess as sp

from openeye.oechem import *
from openforcefield.typing.engines.smirnoff import *
from openforcefield.utils import mergeStructure
import parmed as pmd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
dummy_atoms = pmd.amber.AmberParm('generated/a-bam-p/dummy.prmtop', xyz='generated/a-bam-p/dummy.inpcrd')
action = pmd.tools.checkValidity(dummy_atoms)
action.execute()

dummy_atoms

generated/a-bam-p/dummy.prmtop looks OK to me!


<AmberParm 3 atoms; 3 residues; 0 bonds; NOT parametrized>

Test that we can write and re-read in the `dummy_atoms` structure.

In [41]:
dummy_atoms.save('dummy.prmtop')
dummy_atoms.save('dummy.inpcrd')

test = pmd.amber.AmberParm('dummy.prmtop', xyz='dummy.inpcrd')
action = pmd.tools.checkValidity(test)
action.execute()
test

dummy.prmtop looks OK to me!


<AmberParm 3 atoms; 3 residues; 0 bonds; NOT parametrized>

Looks good!

In [42]:
host_guest = pmd.amber.AmberParm('generated/a-bam-p/hg.prmtop', xyz='generated/a-bam-p/hg.inpcrd')
action = pmd.tools.checkValidity(host_guest)
action.execute()
host_guest

generated/a-bam-p/hg.prmtop looks OK to me!


<AmberParm 143 atoms; 7 residues; 148 bonds; parametrized>

Test that we can write and re-read the `host-guest` structure.

In [43]:
host_guest.save('host_guest.prmtop')
host_guest.save('host_guest.inpcrd')

test = pmd.amber.AmberParm('host_guest.prmtop', xyz='host_guest.inpcrd')
action = pmd.tools.checkValidity(test)
action.execute()
test

host_guest.prmtop looks OK to me!


<AmberParm 143 atoms; 7 residues; 148 bonds; parametrized>

Okay great. Now let's merge them into a single structure and save.

In [44]:
merged_output = dummy_atoms + host_guest
merged_output.save('merged.prmtop')
merged_output.save('merged.inpcrd')

The `prmtop` is not a valid. `pmemd` says:
```
At line 615 of file prmtop_dat.F90 (unit = 8, file = 'solvated_smirnoff.prmtop')
Fortran runtime error: Bad value during integer read
```
We can also check because ParmEd won't be able to read the file.

In [45]:
merged_input = pmd.amber.AmberParm('merged.prmtop', xyz='merged.inpcrd')
action = pmd.tools.checkValidity(merged_input)
action.execute()
merged_input

IndexError: list index out of range

In [46]:
!cat dummy.prmtop

%VERSION  VERSION_STAMP = V0001.000  DATE = 02/13/18  21:09:49
%FLAG TITLE
%FORMAT(20a4)
default_name
%FLAG POINTERS
%FORMAT(10I8)
       3       1       0       0       0       0       0       0       0       0
       3       3       0       0       0       0       0       0       1       0
       0       0       0       0       0       0       0       0       1       0
       0
%FLAG ATOM_NAME
%FORMAT(20a4)
Pb  Pb  Pb  
%FLAG CHARGE
%FORMAT(5E16.8)
  0.00000000E+00  0.00000000E+00  0.00000000E+00
%FLAG ATOMIC_NUMBER
%FORMAT(10I8)
      15      15      15
%FLAG MASS
%FORMAT(5E16.8)
  2.10000000E+02  2.10000000E+02  2.10000000E+02
%FLAG ATOM_TYPE_INDEX
%FORMAT(10I8)
       1       1       1
%FLAG NUMBER_EXCLUDED_ATOMS
%FORMAT(10I8)
       1       1       1
%FLAG NONBONDED_PARM_INDEX
%FORMAT(10I8)
       1
%FLAG RESIDUE_LABEL
%FORMAT(20a4)
DUM DUM DUM 
%FLAG RESIDUE_POINTER
%FORMAT(10I8)
       1       2       3
%FLAG BOND_FORCE_CONSTANT
%FORMAT(5E16.8)

%FLAG BOND_EQUIL_VALUE
%FORMAT(5

In [48]:
!cat host_guest.prmtop

%VERSION  VERSION_STAMP = V0001.000  DATE = 02/13/18  21:10:54
%FLAG TITLE
%FORMAT(20a4)

%FLAG POINTERS
%FORMAT(10I8)
     143     143      72      76     165     111     304     202       0       0
     845       7      76     111     202       6       4     420       1       0
       0       0       0       0       0       0       0       0      21       0
       0
%FLAG ATOM_NAME
%FORMAT(20a4)
C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  C6  H61 H62 O6  
HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  C6  H61 H62 
O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  C6  H61 
H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  C6  
H61 H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  
C6  H61 H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  
O5  C6  H61 H62 O6  HO6 N1  H1  H2  HN11C1  H3  H4  C2  H5  H6  C3  H7  H8  C4  
H9  H10 H11 
%FLAG CHARGE
%FORMA

  1.38998893E+04  5.85549272E+02  8.66220817E+01  4.95732238E+02  1.38998893E+04
  5.85549272E+02  1.04986921E+02  5.85549272E+02  1.04986921E+02  1.38998893E+04
  5.85549272E+02  8.66220817E+01  8.66220817E+01  4.95732238E+02  1.38998893E+04
  5.85549272E+02  1.04986921E+02  1.38998893E+04  5.85549272E+02  8.66220817E+01
  4.95732238E+02  1.38998893E+04  5.85549272E+02  8.66220817E+01  4.95732238E+02
  1.38998893E+04  5.85549272E+02  1.04986921E+02  5.85549272E+02  1.04986921E+02
  1.38998893E+04  5.85549272E+02  8.66220817E+01  8.66220817E+01  4.95732238E+02
  1.40653836E+04  3.41947622E+03  1.31072000E+05  1.40653836E+04  3.06218680E+03
  1.38998893E+04  1.31072000E+05  1.40653836E+04  3.06218680E+03  1.38998893E+04
  1.31072000E+05  1.40653836E+04  3.41947622E+03  1.40653836E+04  3.41947622E+03
  1.31072000E+05  1.40653836E+04  3.06218680E+03  3.06218680E+03  1.38998893E+04
  1.31072000E+05  1.40653836E+04  3.41947622E+03  1.31072000E+05  1.40653836E+04
  3.06218680E+03  1.38998893

In [49]:
!cat merged.prmtop

%VERSION  VERSION_STAMP = V0001.000  DATE = 02/13/18  21:11:30
%FLAG TITLE
%FORMAT(20a4)
default_name
%FLAG POINTERS
%FORMAT(10I8)
     146     143      72      76     165     111     304     202       0       0
     848      10      76     111     202       6       4     420       1       0
       0       0       0       0       0       0       0       0      21       0
       0
%FLAG ATOM_NAME
%FORMAT(20a4)
Pb  Pb  Pb  C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  C6  
H61 H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  
C6  H61 H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  
O5  C6  H61 H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  
H5  O5  C6  H61 H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  
C5  H5  O5  C6  H61 H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  
H4  C5  H5  O5  C6  H61 H62 O6  HO6 N1  H1  H2  HN11C1  H3  H4  C2  H5  H6  C3  
H7  H8  C4  H9  H10 